In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2
import os
import time

print(tf.__version__)
from tensorflow.keras import optimizers, metrics, losses
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
from keras.initializers import RandomNormal

from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
from tensorflow.keras.layers import Conv1D,MaxPooling1D,Conv1DTranspose,AveragePooling1D,GlobalMaxPool1D,GlobalAveragePooling1D
from tensorflow.keras.layers import Conv2D,MaxPool2D,Conv2DTranspose,AveragePooling2D,GlobalMaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv3D,MaxPooling3D,Conv3DTranspose,AveragePooling3D,GlobalMaxPool3D,GlobalAveragePooling3D
from tensorflow.keras.layers import Dense,Flatten,Dropout,Concatenate,Layer,BatchNormalization,Input,Add,Activation,Average

from tensorflow.keras import layers
import tensorflow as tf


2.10.1


In [2]:
#data_path = "outlineOutput"
#data_path = "H:\\download\\blender\\projects\\vdmTests\\outlineOutput"
#output_path = "H:\\tmp\\dexined"

data_path = "I:\\tmp\\datasets\\tinyDexinedDataset2"
output_path = "I:\\tmp\\teed"

train_path = data_path + "train"
edge_train_path = data_path + "train_edge"

test_path = data_path + "test"
edge_test_path = data_path + "test_edge"

val_path = data_path + "val"
edge_val_path = data_path + "val_edge"

print(edge_test_path)

I:\tmp\datasets\tinyDexinedDataset2test_edge


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BUFFER_SIZE = 1024

img_shape =None

class DataLoader(tf.keras.utils.Sequence):

    def __init__(self,data_dir, list_name, data_name, image_width, image_height, batch_size, shuffle,crop_img=False):
        self.base_dir = data_dir
        self.list_name = list_name
        self.dim_w = image_width
        self.dim_h = image_height
        self.data_name =data_name
        self.bs = batch_size
        self.shuffle=shuffle
        self.crop_img = crop_img
        self.data_list = self._build_index()
        self.on_epoch_end()


    def _build_index(self):

        file_path = os.path.join(self.base_dir, self.list_name)
        with open(file_path,'r') as f:
            file_list = f.readlines()
        file_list = [line.strip() for line in file_list] # to clean the '\n'
        file_list = [line.split(' ') for line in file_list] # separate paths

        input_path = [os.path.join(
            self.base_dir,line[0]) for line in file_list]
        gt_path = [os.path.join(
            self.base_dir,line[1]) for line in file_list]
        
        sample_indeces= [input_path, gt_path]
        return sample_indeces

    def on_epoch_end(self):
        self.indices = np.arange(len(self.data_list[0]))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return len(self.indices)//self.bs


    def __getitem__(self, index):
        print("__getitem__",index)
        indices = self.indices[index*self.bs:(index+1)*self.bs]

        x_list,y_list = self.data_list
        tmp_x_path = [x_list[k] for k in indices]
        tmp_y_path = [y_list[k] for k in indices]

        x,y = self.__data_generation(tmp_x_path,tmp_y_path)

        return x,y

    def __data_generation(self,x_path,y_path):

        x = np.empty((self.bs, self.dim_h, self.dim_w, 3), dtype="float32")
        y = np.empty((self.bs, self.dim_h, self.dim_w, 1), dtype="float32")

        for i,tmp_data in enumerate(x_path):
            tmp_x_path = tmp_data
            tmp_y_path = y_path[i]
            tmp_x,tmp_y = self.transformer(tmp_x_path,tmp_y_path)
            x[i,]=tmp_x
            y[i,]=tmp_y
        print(x.shape)
        print(y.shape)
        return x,y

    def transformer(self, x_path, y_path):
        tmp_x = cv2.imread(x_path)
        tmp_y = cv2.imread(y_path,cv2.IMREAD_GRAYSCALE)

        h,w,_ = tmp_x.shape

        if self.crop_img:
            i_h = random.randint(0,h-self.dim_h)
            i_w = random.randint(0,w-self.dim_w)
            tmp_x = tmp_x[i_h:i_h+self.dim_h,i_w:i_w+self.dim_w,]
            tmp_y = tmp_y[i_h:i_h+self.dim_h,i_w:i_w+self.dim_w,]
        else:
            tmp_x = cv2.resize(tmp_x,(self.dim_w,self.dim_h))
            tmp_y = cv2.resize(tmp_y,(self.dim_w,self.dim_h))

        tmp_y = np.expand_dims(np.float32(tmp_y)/255.,axis=-1)
        tmp_x = np.float32(tmp_x)
        return tmp_x, tmp_y


def preprocess(x,y,H=512, W=912):

    def f(x,y):
        x = x.decode()
        y = y.decode()

        x = read_image(x)
        y = read_edge(y)
        return x, y

    images, edges = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    images.set_shape([H, W, 3])
    edges.set_shape([H, W, 1])
    return images, edges

def tf_data(x,y,bs):
    data = tf.data.Dataset.from_tensor_slices((x,y))
    data = data.shuffle(buffer_size=bs)
    data = data.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    data = data.batch(12)
    data = data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return data



In [3]:
#data_path = "outlineOutput"
#data_path = "H:\\download\\blender\\projects\\vdmTests\\outlineOutput"
#output_path = "H:\\tmp\\dexined"

data_path = "I:\\tmp\\datasets\\tinyDexinedDataset2"
output_path = "I:\\tmp\\teed"

train_path = data_path + "/train/*"
edge_train_path = data_path + "/train_edge/*"

test_path = data_path + "/test/*"
edge_test_path = data_path + "/test_edge/*"

val_path = data_path + "/val/*"
edge_val_path = data_path + "/val_edge/*"

print(edge_test_path)

def load_data(ipath, epath):
    images = sorted(glob(os.path.join(ipath)))
    edges = sorted(glob(os.path.join(epath)))
    return images, edges

images, edges = load_data(train_path, edge_train_path)
valimg, valedg = load_data(val_path, edge_val_path)
print(len(images), len(valimg))

def read_image(path, H=12, W=12):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W,H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_edge(path, H=12, W=12):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W,H))
    x = np.expand_dims(x, axis=-1)
    x = x/255.0
    x = x.astype(np.float32)
    return x

def preprocess(x,y,H=12, W=12):

    def f(x,y):
        x = x.decode()
        y = y.decode()

        x = read_image(x)
        y = read_edge(y)
        return x, y

    images, edges = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    images.set_shape([H, W, 3])
    edges.set_shape([H, W, 1])
    return images, edges

def tf_data(x,y,bs):
    data = tf.data.Dataset.from_tensor_slices((x,y))
    data = data.shuffle(buffer_size=bs)
    data = data.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    data = data.batch(12)
    data = data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return data

train_data = tf_data(images, edges, len(images))
val_data = tf_data(valimg, valedg, len(valimg))

print(len(train_data),len(val_data))

I:\tmp\datasets\tinyDexinedDataset2/test_edge/*
4 2
1 1


In [4]:

def train(model,model_name,train_data,test_data,lr,beta1,max_epochs,batch_size):
    # Validation and Train dataset generation

    train_data = train_data
    n_train =len(train_data) #data_cache["n_files"]
    val_data = test_data
    
    # Summary and checkpoint manager
    model_dir = model_name
    summary_dir = os.path.join(output_path,'logs',model_dir)
    train_log_dir=os.path.join(summary_dir,'train')
    val_log_dir =os.path.join(summary_dir,'test')

    checkpoint_dir = os.path.join(output_path,"checkpoint_dir",model_dir)
    epoch_ckpt_dir = checkpoint_dir + 'epochs'
    os.makedirs(epoch_ckpt_dir, exist_ok=True)
    os.makedirs(train_log_dir,exist_ok=True)
    os.makedirs(val_log_dir,exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)

    train_writer = tf.summary.create_file_writer(train_log_dir)
    val_writer = tf.summary.create_file_writer(val_log_dir)

    my_model = model

    # accuracy = metrics.SparseCategoricalAccuracy()
    accuracy = metrics.BinaryAccuracy()
    accuracy_val = metrics.BinaryAccuracy()
    loss_bc = losses.BinaryCrossentropy()
    optimizer = optimizers.Adam(
        learning_rate=lr, beta_1=beta1)
    iter = 0

    imgs_res_folder = os.path.join(output_path,model_dir, "current_training")
    os.makedirs(imgs_res_folder, exist_ok=True)
    global_loss = 1000.
    t_loss = []
    ckpt_save_mode = "h5"
    tmp_lr = lr
    for epoch in range(max_epochs):
        # training
        t_loss = []
        # if epoch in self.args.adjust_lr:
        tmp_lr=tmp_lr*0.1
        optimizer.lr.assign(tmp_lr)
        for step, (x, y) in enumerate(train_data):

            with tf.GradientTape() as tape:
                pred = my_model(x, training=True)

                preds, loss = pre_process_binary_cross_entropy(
                    loss_bc, pred, y, use_tf_loss=False)

            accuracy.update_state(y_true=y, y_pred=preds[-1])
            gradients = tape.gradient(loss, my_model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, my_model.trainable_variables))

            # logging the current accuracy value so far.
            t_loss.append(loss.numpy())
            if step % 10 == 0:
                print("Epoch:", epoch, "Step:", step, "Loss: %.4f" % loss.numpy(),
                        "Accuracy: %.4f" % accuracy.result(), time.ctime())

            #if step % 10 == 0:
            #    # visualize preds
            #    img_test = 'Epoch: {0} Sample {1}/{2} Loss: {3}' \
            #        .format(epoch, step, n_train // batch_size, loss.numpy())
            #    vis_imgs = visualize_result(
            #        x=x[2], y=y[2], p=preds, img_title=img_test)
            #    cv.imwrite(os.path.join(imgs_res_folder, 'results.png'), vis_imgs)
            if step % 20 == 0 and loss < global_loss:  # 500
                if epoch==0 and step==0:
                    tmp_loss = np.array(t_loss)
                    with train_writer.as_default():
                        tf.summary.scalar('loss', tmp_loss.mean(), step=epoch)
                        tf.summary.scalar('accuracy', accuracy.result(), step=epoch)

                save_ckpt_path = os.path.join(checkpoint_dir, "DexiNedL_model.h5")
                Model.save_weights(my_model, save_ckpt_path, save_format='h5')

                global_loss = loss
                print("Model saved in:  ", save_ckpt_path, "Current loss:", global_loss.numpy())

            iter += 1  # global iteration

        t_loss = np.array(t_loss)
        # train summary
        if epoch!=0:
            with train_writer.as_default():
                tf.summary.scalar('loss', t_loss.mean(), step=epoch)
                tf.summary.scalar('accuracy', accuracy.result(), step=epoch)

        Model.save_weights(my_model, os.path.join(epoch_ckpt_dir, "DexiNed{}_model.h5".format(str(epoch))),
                            save_format=ckpt_save_mode)
        print("Epoch:", epoch, "Model saved in Loss: ", t_loss.mean())

        # validation
        t_val_loss = []
        for i, (x_val, y_val) in enumerate(val_data):

            pred_val = my_model(x_val)
            v_logits, V_loss = pre_process_binary_cross_entropy(
                loss_bc, pred_val, y_val, use_tf_loss=False)
            accuracy_val.update_state(y_true=y_val, y_pred=v_logits[-1])
            t_val_loss.append(V_loss.numpy())
            if i == 7:
                break
        val_acc = accuracy_val.result()
        t_val_loss = np.array(t_val_loss)
        print("Epoch(validation):", epoch, "Val loss: ", t_val_loss.mean(),
                "Accuracy: ", val_acc.numpy())
        # validation summary
        with val_writer.as_default():
            tf.summary.scalar('loss', t_val_loss.mean(), step=epoch)
            tf.summary.scalar('accuracy', val_acc.numpy(), step=epoch)

        # Reset metrics every epoch
        accuracy.reset_states()
        accuracy_val.reset_states()

    my_model.summary()


In [5]:
@tf.function
def smish(input):
    return input * tf.math.tanh(tf.math.log(1+tf.math.sigmoid(input)))

class Smish(layers.Layer):

    def __init__(self):
        """
        Init method.
        """
        super().__init__()

    def call(self, input):
        """
        Forward pass of the function.
        """
        return smish(input)


In [ ]:

weight_init = tf.initializers.glorot_uniform()

l2 = regularizers.l2
w_decay=1e-3

glorot_normal = RandomNormal(stddev=0.01)

orthogonal = tf.keras.initializers.Orthogonal(
    gain=1.0, seed=None
)

self_conv2D_1 = Conv2D(16 , kernel_size=(3,3),strides=(2,2),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_conv2D_11 = Conv2D(16 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_conv2D_17 = Conv2D(32 , kernel_size=(3,3),strides=(1,1),padding = 'same')
self_conv2D_18 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True,kernel_initializer= glorot_normal)
self_conv2D_19 = Conv2D(32 , kernel_size=(1,1),strides=(2,2),padding = 'same')
self_conv2D_12 = Conv2D(32 , kernel_size=(3,3),strides=(1,1),padding = 'same')
self_transpoze2D_1 = Conv2DTranspose(1 , kernel_size=(2,2),strides=(2,2),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_conv2D_5 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True,kernel_initializer= glorot_normal)
self_maxpool2D_1 = MaxPool2D(pool_size=(3,3),strides=(2,2),padding = 'same')
self_conv2D_24 = Conv2D(48 , kernel_size=(1,1),strides=(1,1),padding = 'same')
self_transpoze2D_4 = Conv2DTranspose(1 , kernel_size=(2,2),strides=(2,2),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_conv2D_2 = Conv2D(32 , kernel_size=(3,3),strides=(1,1),padding = 'same')
self_conv2D_3 = Conv2D(48 , kernel_size=(3,3),strides=(1,1),padding = 'same')
self_conv2D_20 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True,kernel_initializer= glorot_normal)
self_transpoze2D_2 = Conv2DTranspose(16 , kernel_size=(2,2),strides=(2,2),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_conv2D_21 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True,kernel_initializer= glorot_normal)
self_transpoze2D_3 = Conv2DTranspose(1 , kernel_size=(4,4),strides=(2,2),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_concatenate_1 = Concatenate(axis=3)
self_conv2D_22 = Conv2D(24 , kernel_size=(3,3),strides=(1,1),padding = 'same')
self_conv2D_23 = Conv2D(24 , kernel_size=(3,3),strides=(1,1),padding = 'same')
self_conv2D_6 = Conv2D(24 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu')
self_concatenate_2 = Concatenate(axis=3)
self_conv2D_7 = Conv2D(24 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu')
self_concatenate_3 = Concatenate(axis=3)
self_conv2D_8 = Conv2D(24 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu')
self_concatenate_4 = Concatenate(axis=3)
self_conv2D_4 = Conv2D(24 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu')
self_conv2D_10 = Conv2D(32 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu',kernel_initializer= orthogonal)
self_conv2D_13 = Conv2D(32 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu')
self_concatenate_5 = Concatenate(axis=3)
self_conv2D_14 = Conv2D(32 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu')
self_concatenate_6 = Concatenate(axis=3)
self_conv2D_15 = Conv2D(32 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu')
self_concatenate_7 = Concatenate(axis=3)
self_conv2D_16 = Conv2D(32 , kernel_size=(3,3),strides=(1,1),padding = 'same',activation='relu')
self_conv2D_9 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',kernel_initializer= orthogonal)

self_activation_9=Smish()
self_activation_2=Smish()
self_activation_3=Smish()
self_activation_8=Smish()
self_activation_7=Smish()
self_activation_5=Smish()
self_activation_1=Smish()
self_activation_6=Smish()
self_activation_4=Smish()

x = Input(shape=(12,12,3))


conv2D_1 = self_conv2D_1(x)


activation_9 = self_activation_9(conv2D_1)
conv2D_11 = self_conv2D_11(activation_9)
activation_2 = self_activation_2(conv2D_11)
conv2D_17 = self_conv2D_17(activation_2)
conv2D_18 = self_conv2D_18(activation_2)
conv2D_19 = self_conv2D_19(activation_2)
activation_3 = self_activation_3(conv2D_17)
activation_8 = self_activation_8(conv2D_18)
conv2D_12 = self_conv2D_12(activation_3)
transpoze2D_1 = self_transpoze2D_1(activation_8)
conv2D_5 = self_conv2D_5(conv2D_12)
maxpool2D_1 = self_maxpool2D_1(conv2D_12)
activation_7 = self_activation_7(conv2D_5)
add_1 = Add()([conv2D_19,maxpool2D_1])
conv2D_24 = self_conv2D_24(maxpool2D_1)
transpoze2D_4 = self_transpoze2D_4(activation_7)
conv2D_2 = self_conv2D_2(add_1)
activation_5 = self_activation_5(conv2D_2)
conv2D_3 = self_conv2D_3(activation_5)
average_1 = Average()([conv2D_3,conv2D_24])
conv2D_20 = self_conv2D_20(average_1)
activation_1 = self_activation_1(conv2D_20)
transpoze2D_2 = self_transpoze2D_2(activation_1)
conv2D_21 = self_conv2D_21(transpoze2D_2)
activation_6 = self_activation_6(conv2D_21)
transpoze2D_3 = self_transpoze2D_3(activation_6)
concatenate_1_TMP = [transpoze2D_1,transpoze2D_3,transpoze2D_4]
concatenate_1 = self_concatenate_1(concatenate_1_TMP)
conv2D_22 = self_conv2D_22(concatenate_1)
conv2D_23 = self_conv2D_23(conv2D_22)
activation_4 = self_activation_4(conv2D_23)
conv2D_6 = self_conv2D_6(activation_4)
concatenate_2_TMP = [conv2D_23,conv2D_6]
concatenate_2 = self_concatenate_2(concatenate_2_TMP)
conv2D_7 = self_conv2D_7(concatenate_2)
concatenate_3_TMP = [concatenate_2,conv2D_7]
concatenate_3 = self_concatenate_3(concatenate_3_TMP)
conv2D_8 = self_conv2D_8(concatenate_3)
concatenate_4_TMP = [concatenate_2,concatenate_3,conv2D_8]
concatenate_4 = self_concatenate_4(concatenate_4_TMP)
conv2D_4 = self_conv2D_4(concatenate_4)


add_2 = Add()([conv2D_23,conv2D_4])


conv2D_10 = self_conv2D_10(add_2)
conv2D_13 = self_conv2D_13(conv2D_10)
concatenate_5_TMP = [conv2D_10,conv2D_13]
concatenate_5 = self_concatenate_5(concatenate_5_TMP)
conv2D_14 = self_conv2D_14(concatenate_5)
concatenate_6_TMP = [concatenate_5,conv2D_14]
concatenate_6 = self_concatenate_6(concatenate_6_TMP)
conv2D_15 = self_conv2D_15(concatenate_6)
concatenate_7_TMP = [concatenate_5,concatenate_6,conv2D_15]
concatenate_7 = self_concatenate_7(concatenate_7_TMP)
conv2D_16 = self_conv2D_16(concatenate_7)
add_3 = Add()([conv2D_10,conv2D_16])
conv2D_9 = self_conv2D_9(add_3)

#outputs = tf.nn.depth_to_space(conv2D_9, 1)
model = keras.Model(inputs =x , outputs=conv2D_9)


In [ ]:
def pre_process_binary_cross_entropy(bc_loss,input, label, use_tf_loss=False):
    # preprocess data
    y = label
    loss = 0
    w_loss=1.0
    preds = []
    
    #print(label.shape)
    for tmp_p in input:
        # tmp_p = input[i]

        # loss processing
        tmp_y = tf.cast(y, dtype=tf.float32)
        mask = tf.dtypes.cast(tmp_y > 0., tf.float32)
        b,h,w,c=mask.get_shape()
        positives = tf.math.reduce_sum(mask, axis=[1, 2, 3], keepdims=True)
        negatives = h*w*c-positives

        beta2 = (1.*positives) / (negatives + positives) # negatives in hed
        beta = (1.1*negatives)/ (positives + negatives) # positives in hed
        pos_w = tf.where(tf.equal(y, 0.0), beta2, beta)
        logits = tf.sigmoid(tmp_p)

        l_cost = bc_loss(y_true=tmp_y, y_pred=logits,
                         sample_weight=pos_w)

        preds.append(logits)
        loss += (l_cost*w_loss)

    return preds, loss

In [ ]:
#train_data = DataLoader(data_path, "train_list.txt", "blender1", 12, 12, 4, True)
# val_data = DataLoader(data_path, "val_list.txt", "blender2", 12, 12, 4, True)

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam", run_eagerly=True)

train(model,"dexined_try",train_data,val_data,0.0001,0.5,11,12)